In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('toy_data.csv') # or exchange with your generated data 'your_data.csv'

# Shift time, starting from 1, add terms relevant for the model
df.loc[:, "t"] = df.loc[:, "t"]+1
df["is_discussion"] = (df["t"] != 1).astype(int)
df["is_responder"] = (df["is_initiator"] != 1).astype(int)

# Add x_0 column for anchoring effect
for i in range(len(df)):
    if df.loc[i, "t"] == 1:
        df.loc[i, "x_0"] = df.loc[i, "x_j"]

    else:
        df.loc[i, "x_0"] = df.loc[i-2, "x_0"]

In [ ]:
# Prepare data for model fitting
delta_x = df['dx'].to_numpy()

x_i = df['x_i'].to_numpy()
H_i = df["H_i"].to_numpy()
x_j = df['x_j'].to_numpy()
H_j = df['H_j'].to_numpy()

t_data = df['t'].to_numpy()
d_data = df['delta'].to_numpy()

is_initiator = df['is_initiator'].to_numpy()
is_responder = df['is_responder'].to_numpy()

x_0 = df['x_0'].to_numpy()
is_discussion = df['is_discussion'].to_numpy()

In [ ]:
##### Full model  #####

In [ ]:
with pm.Model() as full_model:
    alpha = pm.Normal("alpha", sigma=1.0)
    tau = pm.HalfNormal("tau", sigma=1.0)

    beta_t = pm.HalfNormal("beta_t", sigma=1)    
    #beta_t = pm.Deterministic("beta_t", beta +0.01) 
    b_t = pm.Uniform("b_t", lower=-2, upper=2)


    beta_a = pm.HalfNormal("beta_a", sigma=1.0)
    a = pm.Uniform("a", lower=-2, upper=2)

    beta_c = pm.Normal("beta_c", sigma=1)
    
    sigma0 = pm.HalfNormal("sigma0", sigma=1)
    gamma = pm.HalfNormal("gamma", sigma=1)
    eps = pm.HalfNormal("eps", sigma=1)
    
    # Interaction term
    interaction = alpha * pm.math.exp(-t_data / tau )

    # Mean opinion shift
    mu = interaction * ( x_j - x_i  )  + beta_t * (d_data * b_t - x_i) + beta_a * (a - x_i) + beta_c * (x_0 - x_i) * is_responder
    
    # Std term
    sigma = sigma0 + gamma * mu**2 + eps * H_i * is_discussion
    
    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=delta_x)
    
    # Posterior sampling
    trace = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=42, nuts_sampler='nutpie') 

# Save trace
az.to_netcdf(trace, f"traces/full_model_trace_gpt.nc")

In [ ]:
# alpha: interaction strength, tau: time decay, beta_t: topic bias strength, b_t: topic bias attractor, beta_a: agreement bias strength, a: agreement bias attractor, beta_c: anchor bias strength

pm.summary(trace, var_names=["alpha", "tau", "beta_t", "b_t", "beta_a", "a", "beta_c", "sigma0", "gamma", "eps"])

In [ ]:
##### Model without interaction term  #####

In [ ]:
with pm.Model() as model_no_interaction:
    #alpha = pm.Normal("alpha", sigma=1.0)
    #tau = pm.HalfNormal("tau", sigma=1.0)

    beta_t = pm.HalfNormal("beta_t", sigma=1)    
    b_t = pm.Uniform("b_t", lower=-2, upper=2)


    beta_a = pm.HalfNormal("beta_a", sigma=1.0)
    a = pm.Uniform("a", lower=-2, upper=2)

    beta_c = pm.Normal("beta_c", sigma=1)
    
    sigma0 = pm.HalfNormal("sigma0", sigma=1)
    gamma = pm.HalfNormal("gamma", sigma=1)
    eps = pm.HalfNormal("eps", sigma=1)
    
    # Interaction term
    #interaction = alpha * pm.math.exp(-t_data / tau)

    # Mean opinion shift
    mu = beta_t * (d_data * b_t - x_i) + beta_a * (a - x_i) + beta_c * (x_0 - x_i) * is_responder
    
    # Std term
    sigma = sigma0 + gamma * mu**2 + eps * H_i * is_discussion
    
    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=delta_x)
    
    # Posterior sampling
    trace_2 = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=42, nuts_sampler='nutpie')

# Save trace
az.to_netcdf(trace_2, f"traces/model_no_int_trace_gpt.nc")

In [ ]:
# beta_t: topic bias strength, b_t: topic bias attractor, beta_a: agreement bias strength, a: agreement bias attractor, beta_c: anchor bias strength

pm.summary(trace_2, var_names=["beta_t", "b_t", "beta_a", "a", "beta_c", "sigma0", "gamma", "eps"])

In [ ]:
# Compare models

# Calculate log likelihoods

with full_model:
    pm.compute_log_likelihood(trace)

with model_no_interaction:
    pm.compute_log_likelihood(trace_2)

df_comp = az.compare({"full_model": trace, "no interaction": trace_2})
az.plot_compare(df_comp, figsize=(4,3))

In [ ]:
# Check pareto k diagnostics - should be < 0.7 for > 99% of data points

az.loo(trace, pointwise=True)

In [ ]:
az.loo(trace_2, pointwise=True)